In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
path = '..\\Data\\'
rs = [3, 13, 42, 57, 81]
random_state = 42
n_features_to_select = 5
n_runs = 100
df_path = path + 'predictions_cls_seed{}_feat{}_runs{}.csv'.format(random_state, n_features_to_select, n_runs)
rfe_path = path + 'rfe_cls_res_seed{}_feat{}_runs{}.pkl'.format(random_state, n_features_to_select, n_runs)

In [3]:
def classifier(x):
    x = int(x)
    if x == 0:
        return 'ea'
    if x == 1:
        return 'ma'
    if x == 2:
        return 'mi'
    return 'ei'

In [4]:
results = pd.read_csv(df_path, header=0, sep=';', decimal=',', index_col=0)

In [5]:
results[results.columns[:-1]] = results[results.columns[:-1]].applymap(classifier)

In [6]:
results.describe()

,accuracy
count,101.000000
mean,0.376238
std,0.210361
min,0.000000
25%,0.166667
50%,0.333333
75%,0.500000
max,0.666667


In [7]:
results[results.index != 'inhibition efficiency ZE41 / %'].mode()

,0,5,36,45,13,54,accuracy
0,ea,ei,mi,mi,mi,ei,0.166667


In [8]:
results[results.index == 'inhibition efficiency ZE41 / %']

,0,5,36,45,13,54,accuracy
inhibition efficiency ZE41 / %,ea,mi,mi,ma,mi,ma,0.0


In [9]:
with open(rfe_path, 'rb') as f:
    rfe_dict = pickle.load(f) #counts, vals
counts = rfe_dict['counts']
vals = rfe_dict['vals']

In [10]:
vals[np.argmax(counts)]

array(['P_VSA_MR_5', 'Mor14u', 'Mor04m', 'HOMO / eV', 'LUMO / eV'],
      dtype='<U14')

In [11]:
np.max(counts)

3

In [12]:
result = []
indiv=[]

In [13]:
#res_dict

In [14]:
np.where(counts > 1)[0]

array([36, 42, 45, 47, 56], dtype=int64)

In [15]:
for idx in np.where(counts > 1)[0]:
    print(counts[idx], vals[idx])

2 ['P_VSA_MR_5' 'Mor04m' 'Mor14s' 'H3m' 'LUMO / eV']
3 ['P_VSA_MR_5' 'Mor14u' 'Mor04m' 'HOMO / eV' 'LUMO / eV']
3 ['P_VSA_MR_5' 'Mor14u' 'Mor04m' 'Mor29v' 'LUMO / eV']
2 ['P_VSA_MR_5' 'Mor14u' 'Mor19m' 'Mor14s' 'LUMO / eV']
2 ['P_VSA_MR_5' 'P_VSA_e_3' 'Mor19m' 'H3m' 'LUMO / eV']


In [16]:
for rand in rs:
    rfe_path = path + 'rfe_cls_res_seed{}_feat{}_runs{}.pkl'.format(rand, n_features_to_select, n_runs)
    with open(rfe_path, 'rb') as f:
        rfe_dict = pickle.load(f) #counts, vals
    counts = rfe_dict['counts']
    vals = rfe_dict['vals']
    if rand != random_state:
        df_path = path + 'predictions_seed{}_feat{}_runs{}.csv'.format(rand, n_features_to_select, n_runs)
        with open(df_path, 'rb') as f:
            df_n = pd.read_csv(df_path, header=0, sep=';', decimal=',', index_col=0, skiprows=1)
            df_n.columns = results.columns
            results = results.append(df_n, ignore_index=True)
    for idx in np.where(counts > 0)[0]:
        result.append([rand, counts[idx], vals[idx]])
        indiv.append('.'.join(sorted(vals[idx])))

In [17]:
results.drop(labels=0, axis=0).describe()

,accuracy
count,500.000000
mean,0.107992
std,0.166975
min,0.000000
25%,0.011572
50%,0.058173
75%,0.083344
max,0.666667


In [18]:
result_df = pd.DataFrame(result, columns=['Seed', 'Count', 'Features'])

In [19]:
result_df

,Seed,Count,Features
0,3,1,"[GATS1i, P_VSA_MR_5, Mor32m, Mor14s, LUMO / eV]"
1,3,1,"[GATS5v, P_VSA_MR_5, Mor14s, H3m, LUMO / eV]"
2,3,1,"[GATS5v, P_VSA_MR_5, Mor14s, HOMO / eV, LUMO /..."
3,3,1,"[GATS5v, P_VSA_MR_5, TDB03m, Mor14s, LUMO / eV]"
4,3,1,"[GGI4, P_VSA_MR_5, Mor14u, Mor04m, LUMO / eV]"
...,...,...,...
454,81,1,"[VE1sign_RG, Mor27e, H3m, HOMO / eV, LUMO / eV]"
455,81,1,"[VE1sign_RG, RDF020i, Mor14s, H3m, LUMO / eV]"
456,81,1,"[X1A, SpMAD_L, P_VSA_MR_5, LUMO / eV, Hlgap / eV]"
457,81,1,"[X4sol, P_VSA_MR_5, H3m, LUMO / eV, Hlgap / eV]"


In [20]:
s = result_df['Features'].str.join('|').str.get_dummies()
s.sum(axis=0).sort_values()[-10:]

Hlgap / eV     49
VE1sign_RG     49
Mor04m         83
Mor29v         99
HOMO / eV     112
Mor14u        142
H3m           168
Mor14s        187
P_VSA_MR_5    342
LUMO / eV     431
dtype: int64

In [21]:
s

,AVS_B(e),DLS_04,DLS_cons,E1i,E1p,E2e,E2m,E2s,E3e,E3m,...,TDB04s,TDB04u,TDB04v,TPSA(NO),VE1sign_RG,VE2_RG,VE2sign_RG,VR2_B(i),X1A,X4sol
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
from collections import Counter

In [23]:
print(len(list(set(indiv))),len(indiv) )

435 459


In [24]:
col_counts = {k: v for k, v in sorted(Counter(indiv).items(), key=lambda item: item[1], reverse=True)}
col_counts

{'H3m.HOMO / eV.Hlgap / eV.LUMO / eV.P_VSA_MR_5': 3,
 'HOMO / eV.LUMO / eV.Mor04m.Mor14s.P_VSA_MR_5': 3,
 'H3m.LUMO / eV.Mor14s.Mor14u.P_VSA_MR_5': 3,
 'H3m.HOMO / eV.LUMO / eV.Mor14s.Mor32m': 2,
 'H0v.H3m.HOMO / eV.LUMO / eV.P_VSA_MR_5': 2,
 'LUMO / eV.Mor04m.Mor14s.Mor29v.P_VSA_MR_5': 2,
 'H3m.HOMO / eV.LUMO / eV.Mor14s.P_VSA_MR_5': 2,
 'H0v.LUMO / eV.Mor14s.Mor14u.P_VSA_MR_5': 2,
 'Hlgap / eV.LUMO / eV.Mor14s.Mor14u.P_VSA_MR_5': 2,
 'LUMO / eV.Mor14s.Mor14u.Mor29v.P_VSA_MR_5': 2,
 'H3m.LUMO / eV.Mor14s.P_VSA_MR_5.P_VSA_e_3': 2,
 'LUMO / eV.Mor14s.Mor29v.P_VSA_MR_5.P_VSA_e_3': 2,
 'HOMO / eV.LUMO / eV.Mor14s.P_VSA_MR_5.RDF020m': 2,
 'HOMO / eV.LUMO / eV.Mor14s.P_VSA_MR_5.TDB03m': 2,
 'HOMO / eV.LUMO / eV.Mor14s.Mor14u.Mor29v': 2,
 'LUMO / eV.Mor14s.Mor14u.P_VSA_MR_5.TDB03m': 2,
 'H3m.HOMO / eV.LUMO / eV.Mor04m.Mor29v': 2,
 'H3m.LUMO / eV.Mor04m.Mor14s.P_VSA_MR_5': 2,
 'H3m.LUMO / eV.Mor04m.Mor14u.P_VSA_MR_5': 2,
 'H3m.HOMO / eV.LUMO / eV.MPC04.P_VSA_MR_5': 2,
 'H3m.LUMO / eV.Mor14s.P